# Демонстрация работы татарской LLM

В этом ноутбуке мы загружаем обученную модель и тестируем её способность генерировать текст на татарском языке.


In [1]:
import torch
import json
from pathlib import Path
from tokenizers import Tokenizer
from model import GPT, GPTConfig


## 1. Загрузка модели и токенизатора


In [2]:
# Пути к модели и токенизатору
model_path = "models/best_model.pt"
tokenizer_path = "tokenizer"

# Определяем устройство
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'

print(f"Используемое устройство: {device}")


Используемое устройство: mps


In [3]:
# Загружаем токенизатор
tokenizer = Tokenizer.from_file(str(Path(tokenizer_path) / "tokenizer.json"))

# Загружаем конфигурацию токенизатора
with open(Path(tokenizer_path) / "config.json", 'r') as f:
    tok_config = json.load(f)

print(f"Размер словаря: {tokenizer.get_vocab_size()}")
print(f"BOS token: {tok_config['bos_token']} (id: {tok_config['bos_token_id']})")
print(f"EOS token: {tok_config['eos_token']} (id: {tok_config['eos_token_id']})")


Размер словаря: 8192
BOS token: <bos> (id: 2)
EOS token: <eos> (id: 3)


In [4]:
# Загружаем чекпоинт модели
checkpoint = torch.load(model_path, map_location=device)

# Создаём конфигурацию модели
model_config = GPTConfig(**checkpoint['config'])

print("Конфигурация модели:")
print(f"  Слои: {model_config.n_layer}")
print(f"  Heads: {model_config.n_head}")
print(f"  Размерность: {model_config.n_embd}")
print(f"  Размер контекста: {model_config.block_size}")
print(f"  Rotary Embeddings: {model_config.use_rotary}")
print(f"\nЛучший val_loss: {checkpoint['best_val_loss']:.4f}")


Конфигурация модели:
  Слои: 6
  Heads: 8
  Размерность: 512
  Размер контекста: 256
  Rotary Embeddings: True

Лучший val_loss: 8.5254


In [5]:
# Создаём и загружаем модель
model = GPT(model_config)
model.load_state_dict(checkpoint['model'])
model.to(device)
model.eval()

print("Модель успешно загружена!")


Number of parameters: 23,109,632 (23.11M)
Модель успешно загружена!


## 2. Функция генерации текста


In [6]:
def generate_text(
    prompt: str,
    max_new_tokens: int = 50,
    temperature: float = 0.8,
    top_k: int = 40,
    num_samples: int = 1
):
    """
    Генерирует текст на основе промпта
    """
    # Токенизируем промпт
    encoded = tokenizer.encode(prompt)
    idx = torch.tensor([encoded.ids], dtype=torch.long, device=device)
    
    results = []
    
    for _ in range(num_samples):
        # Генерируем
        with torch.no_grad():
            generated = model.generate(
                idx,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_k=top_k
            )
        
        # Декодируем
        generated_ids = generated[0].tolist()
        # Убираем специальные токены
        generated_ids = [t for t in generated_ids if t not in [tok_config['bos_token_id'], tok_config['eos_token_id']]]
        text = tokenizer.decode(generated_ids)
        results.append(text)
    
    return results

print("Функция генерации готова!")


Функция генерации готова!


### Тест 1: Продолжение предложения


In [7]:
prompt = "Казан шәһәре"
results = generate_text(prompt, max_new_tokens=30, temperature=0.8, num_samples=3)

print(f"Промпт: {prompt}\n")
for i, text in enumerate(results, 1):
    print(f"Вариант {i}: {text}")
    print("-" * 80)


Промпт: Казан шәһәре

Вариант 1: Казан шәһәре
--------------------------------------------------------------------------------
Вариант 2: Казан шәһәре ,
--------------------------------------------------------------------------------
Вариант 3: Казан шәһәре . “
--------------------------------------------------------------------------------


### Тест 2: Разные промпты


In [8]:
# Примеры вопросов на татарском
prompts = [
    "Татарстанның башкаласы",
    "Татар теле",
    "Казан университеты",
]

for prompt in prompts:
    print(f"\nПромпт: {prompt}")
    print("=" * 80)
    results = generate_text(prompt, max_new_tokens=40, temperature=0.7, num_samples=2)
    for i, text in enumerate(results, 1):
        print(f"Вариант {i}: {text}")
        print("-" * 80)



Промпт: Татарстанның башкаласы
Вариант 1: Татарстанның башкаласы . , ,
--------------------------------------------------------------------------------
Вариант 2: Татарстанның башкаласы дип
--------------------------------------------------------------------------------

Промпт: Татар теле
Вариант 1: Татар теле
--------------------------------------------------------------------------------
Вариант 2: Татар теле һәм
--------------------------------------------------------------------------------

Промпт: Казан университеты
Вариант 1: Казан университеты һәм
--------------------------------------------------------------------------------
Вариант 2: Казан университеты
--------------------------------------------------------------------------------


### Тест 3: Разные формулировки одного вопроса

Проверим, отвечает ли модель последовательно на похожие вопросы.


In [9]:
# Разные формулировки про Казань
prompts = [
    "Казан",
    "Казан шәһәре",
    "Татарстанның башкаласы Казан",
]

for prompt in prompts:
    print(f"\nПромпт: {prompt}")
    print("=" * 80)
    result = generate_text(prompt, max_new_tokens=50, temperature=0.6, num_samples=1)[0]
    print(result)
    print("-" * 80)



Промпт: Казан
Казан
--------------------------------------------------------------------------------

Промпт: Казан шәһәре
Казан шәһәре
--------------------------------------------------------------------------------

Промпт: Татарстанның башкаласы Казан
Татарстанның башкаласы Казан .
--------------------------------------------------------------------------------
